In [7]:
log_root = r'D:\issp_data\features_total'
out_folder = r'C:\Users\howard810804\Desktop\proj_issp'
fraction = 3

model_folder = r'D:\issp_data'
test_size = 0.25

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

from typing import Tuple

import dask.dataframe as dd
import gzip
import numpy as np
import pickle

from distributed.protocol.serialize import register_generic
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tpot import TPOTClassifier
register_generic(TPOTClassifier)

from xgboost import XGBClassifier

In [3]:
def load_data(st: int, ed: int) -> Tuple['np.array']:
    path = os.path.join(log_root, sorted(os.listdir(log_root))[0])
    col_nums = dd.read_csv(path, compression='zip')\
                    .compute()\
                    .shape[-1]
    
    arr_X, arr_y = np.zeros(col_nums).reshape((-1, col_nums)), np.array([0])
    
    for fl in os.listdir(log_root)[(st - 1) * 3: ed * 3]:
        cata = int(fl[31])
        path = os.path.join(log_root, fl)
        tmp = dd.read_csv(path, compression='zip')\
                .compute()\
                .values
        
        arr_X = np.vstack((arr_X, tmp))
        arr_y = np.hstack((arr_y, np.array([cata] * tmp.shape[0])))
        
    return arr_X[1:, ], arr_y[1:]

In [8]:
arr_X, arr_y = load_data(1, 12)
X, y = resample(arr_X, arr_y, n_samples=arr_X.shape[0]//fraction,
                replace=False, stratify=arr_y, random_state=None)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=None)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1981656, 55) (1981656,)
(660552, 55) (660552,)


In [6]:
# # Random forest

# clf_rf = RandomForestClassifier(verbose=2)
# %time clf_rf.fit(X_train, y_train)
# y_pred = clf_rf.predict(X_test)

# print(clf_rf.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(clf_rf.feature_importances_)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
Wall time: 100 ms
0.6641452344931922 

[[274  33  12]
 [ 81  95  24]
 [ 33  39  70]] 

[0.02273802 0.02265463 0.02525208 0.01902999 0.03569033 0.03456387
 0.03628048 0.02736197 0.03487184 0.03888614 0.11772831 0.02757442
 0.04761128 0.03970159 0.05259087 0.08738589 0.01692057 0.02138192
 0.01173042 0.01321383 0.00872564 0.01149711 0.00389506 0.01197418
 0.00965308 0.00281713 0.00916236 0.01562314 0.0150336  0.00589713
 0.01067132 0.01286883 0.01224468 0.01140621 0.00536117 0.01005927
 0.00767197 0.00353355 0.00262316 0.00831709 0.00595099 0.01048866
 0.00842386 0.00339008 0.00212576 0.00365615 0.00336342 0.0013262
 0.00227038 0.00534511 0.00327859 0.00371878 0.00451405 0.00402084
 0.02592304]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [7]:
# # ExtraTrees Classifier

# clf_ext = ExtraTreesClassifier(bootstrap=False, criterion='gini', 
#                                max_features=0.1, min_samples_leaf=3, 
#                                min_samples_split=2, n_estimators=100)

# %time clf_ext.fit(X_train, y_train)
# y_pred = clf_ext.predict(X_test)

# print(clf_ext.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(clf_ext.feature_importances_)

Wall time: 316 ms
0.6459909228441755 

[[298  17   4]
 [121  69  10]
 [ 41  41  60]] 

[0.01555373 0.01420506 0.01599135 0.01658675 0.04164705 0.02867285
 0.02320657 0.01174269 0.03579159 0.05142997 0.06744742 0.02641756
 0.03113275 0.01796577 0.06417587 0.09580206 0.02562008 0.01199064
 0.01211566 0.01573296 0.01543887 0.01672826 0.01011847 0.01375541
 0.01420945 0.00705409 0.01257342 0.01220268 0.01470464 0.01353141
 0.01436605 0.01180101 0.0138908  0.01313389 0.01046144 0.01659115
 0.01257973 0.013035   0.00709733 0.01016099 0.01011469 0.00955905
 0.00852899 0.00945708 0.00796377 0.00557914 0.00456893 0.00718341
 0.00479742 0.00544287 0.00886144 0.01308546 0.00807849 0.00714822
 0.01296857]


In [11]:
# XGBClassifier

# clf_xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#                         colsample_bynode=1, colsample_bytree=1, gamma=0,
#                         learning_rate=0.1, max_delta_step=0, max_depth=3,
#                         min_child_weight=1, missing=None, n_estimators=100, 
#                         n_jobs=1, nthread=None, objective='multi:softprob', 
#                         random_state=0, reg_alpha=0, reg_lambda=1, 
#                         scale_pos_weight=1, seed=None, silent=None, 
#                         subsample=1, verbosity=1)

clf_xgb = XGBClassifier()

%time clf_xgb.fit(X_train, y_train)
y_pred = clf_xgb.predict(X_test)

print(clf_xgb.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')
print(clf_xgb.feature_importances_)

Wall time: 40min 53s
0.7053903402003173 

[[282671  43578   8582]
 [ 64434 110018  18934]
 [ 25219  33858  73258]] 

[0.00330407 0.00476033 0.         0.00342138 0.02871323 0.
 0.01202916 0.01172903 0.05577864 0.08491588 0.23254359 0.03602332
 0.00859231 0.         0.25359854 0.0148772  0.08219305 0.00182651
 0.         0.02720093 0.         0.03114225 0.00904447 0.00725591
 0.02686838 0.01211861 0.00685867 0.00242126 0.002258   0.00152336
 0.00235119 0.         0.00707296 0.00701709 0.         0.
 0.         0.         0.00189595 0.00866268 0.         0.00118199
 0.         0.         0.00189934 0.         0.         0.00595812
 0.00165404 0.         0.         0.         0.         0.
 0.00130855]


In [13]:
md_pa = os.path.join(model_folder, r'model_no-settings.p')
with gzip.GzipFile(md_pa, 'wb') as file:
    pickle.dump(clf_xgb, file)

In [40]:
path = r'D:\issp_data\features_total\s35809_MobileApp_201701_104628_0_features.csv'
df = dd.read_csv(path, sep='\t', compression='zip')
cols = list(df.compute())
cols = cols[0].split(',')

features = clf_xgb.feature_importances_\
                .tolist()

result = [(fea, col) for (fea, col) in zip(features, cols)]
result.sort(reverse=True)
result

[(0.25359854102134705, 'dr_kurtosis'),
 (0.23254358768463135, 'dr_median'),
 (0.0849158763885498, 'dr_mean'),
 (0.0821930468082428, 'OS'),
 (0.05577864497900009, 'dr_max'),
 (0.03602331876754761, 'dr_min'),
 (0.03114224597811699, '世代'),
 (0.028713231906294823, 'ts_std'),
 (0.027200929820537567, '年齢(5才刻み)'),
 (0.026868382468819618, 'メディアターゲット区分'),
 (0.014877201057970524, 'dr_skew'),
 (0.012118610553443432, '未既婚'),
 (0.01202915795147419, 'ts_kurtosis'),
 (0.011729031801223755, 'ts_skew'),
 (0.0090444665402174, '性別'),
 (0.008662676438689232, '同居配偶者年齢(5才刻み)'),
 (0.00859230849891901, 'dr_std'),
 (0.007255910895764828, '性年代'),
 (0.007072962820529938, '最終学歴'),
 (0.007017085328698158, '家族人数'),
 (0.006858667358756065, '職業'),
 (0.005958118010312319, '家族人員(6〜11才(小学生))'),
 (0.004760328680276871, 'ts_mean'),
 (0.003421379718929529, 'ts_min'),
 (0.003304066602140665, 'ts_max'),
 (0.002421256620436907, '職業(詳細)'),
 (0.0023511946201324463, '個人年収'),
 (0.0022580036893486977, '職種'),
 (0.001899339840747416

In [8]:
# config_dict = {
#     'xgboost.XGBClassifier': {
#     }
# }

# clf = TPOTClassifier(generations=3, 
#                      population_size=5, 
#                      n_jobs=-1,
#                      verbosity=2,
#                      config_dict=config_dict,
#                      random_state=None,
#                      use_dask=True)

# %time clf.fit(X_train, y_train)
# print(clf.score(X_test, y_test))
# print(clf)

Generation 1 - Current best internal CV score: 0.6926021107674648
Generation 2 - Current best internal CV score: 0.6926021107674648
Generation 3 - Current best internal CV score: 0.6926021107674648

Best pipeline: XGBClassifier(CombineDFs(input_matrix, input_matrix))
Wall time: 1min 10s
0.6747352496217852
TPOTClassifier(config_dict={'xgboost.XGBClassifier': {}}, crossover_rate=0.1,
               cv=5, disable_update_check=False, early_stop=None, generations=3,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=5,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=True, verbosity=2, warm_start=False)


In [16]:
# best_type = clf.fitted_pipeline_.steps[1]
# print(best_type, '\n')

# %time best_type.fit(X_train, y_train)
# # y_pred = best_type.predict(X_test)

# print(best_type.score(X_test, y_test), '\n')
# # print(confusion_matrix(y_test, y_pred), '\n')
# print(best_type.feature_importances_)

# md_pa = os.path.join(model_folder, r'model_clf2.dat')
# with open(md_pa, 'wb') as file:
#     pickle.dump(best_type, file)

('xgbclassifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)) 



AttributeError: 'tuple' object has no attribute 'fit'

AttributeError: 'tuple' object has no attribute 'score'